In [ ]:
import os
import sys
import pandas as pd
from sklearn import datasets
import statsmodels.api as sm
from stargazer.stargazer import Stargazer
from IPython.display import display, HTML
from sklearn.datasets import load_iris
from pystout import pystout
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
nums = [1, 2, 3, 4, 5]


for i in range(len(nums)):
    for j in range(i + 1, len(nums)):
        print("i", i, nums[i], "j", j, nums[j])

In [ ]:
# Global Settings
pd.set_option("display.float_format", lambda x: "%.4f" % x)

In [ ]:
cwd = os.getcwd()
# Find and import config file
config_path = os.getcwd()

sys.path.append(config_path)
import config

database = config.database
central_banks = config.central_banks
training_data = os.path.join(database, "Training Data")
fed_docs = config.fed_docs
ecb_docs = config.ecb_docs
boe_docs = config.boe_docs

In [ ]:
test = pd.read_csv(
    "/Users/kylenabors/Documents/Database/Models/FinBERT Models/taylor.csv"
)

In [ ]:
sentiment = pd.date_range(start="1/1/1990", end="12/31/2024", freq="D")
sentiment = pd.DataFrame(sentiment, columns=["date"])

In [ ]:
# Import inflation probability data that is stored as a dta file
inflation_prob_us = pd.read_stata(
    "/Users/kylenabors/Documents/Database/Other Data/Inflation Probabilities/USwestimates.dta"
)

inflation_prob_us = inflation_prob_us.rename(columns={"date_ym": "date"})

inflation_prob_eu = pd.read_stata(
    "/Users/kylenabors/Documents/Database/Other Data/Inflation Probabilities/EZwestimates.dta"
)

inflation_prob_eu = inflation_prob_eu.rename(columns={"date_ym": "date"})

inflation_prob = inflation_prob_us.merge(
    inflation_prob_eu, on="date", how="left", suffixes=("_us", "_eu")
)

sentiment = sentiment.merge(inflation_prob, on="date", how="left")

# sentiment = sentiment.ffill()

In [ ]:
inflation_diff = pd.read_excel(
    "/Users/kylenabors/Documents/Database/Market Data/Inflation/12m cumulative inflation.xlsx"
)

print(inflation_diff.dtypes)

sentiment = pd.merge(sentiment, inflation_diff, on="date", how="left")

In [ ]:
indexes = pd.read_excel(
    "/Users/kylenabors/Documents/Database/Other Data/Indexes/five_indexes.xlsx"
)
indexes["date"] = pd.to_datetime(indexes["date"])
sentiment = pd.merge(sentiment, indexes, how="left", left_on="date", right_on="date")

In [ ]:
market = pd.read_csv(f"{database}/Market Data/All Market Data.csv")
market = market.rename(columns={"PCE": "pce", "GDP": "gdp", "GDPPOT": "gdp_pot"})
market["date"] = pd.to_datetime(market["date"])

sentiment = pd.merge(sentiment, market, how="left", left_on="date", right_on="date")

In [ ]:
# # HP Filter

url_map = pd.read_csv(os.path.join(cwd, "url_map.csv"))

for i in range(len(url_map)):
    temp_dates = pd.date_range(start="1/1/1990", end="12/31/2024", freq="D")
    temp_dates = pd.DataFrame(temp_dates, columns=["date"])
    sent = pd.read_csv(url_map["finbert_url"][i])
    sent = sent[["date", "sentiment"]]

    sent["date"] = pd.to_datetime(sent["date"])
    sent = sent.groupby("date").mean().reset_index()

    sent = pd.merge(temp_dates, sent, how="left", left_on="date", right_on="date")

    # Set 'date' as the index for resampling
    sent.set_index("date", inplace=True)

    sent["sentiment"] = sent["sentiment"].ffill()

    # Resample to quarterly frequency
    sent = sent.resample("Q").mean()

    # Drop NaNs
    sent = sent.dropna(subset=["sentiment"])

    # Reset index to turn 'date' back into a column
    sent.reset_index(inplace=True)

    filter_df = sent.copy(deep=True)

    cycle, trend = sm.tsa.filters.hpfilter(filter_df["sentiment"], 1600)

    filter_df["sentiment_cycle"] = cycle
    filter_df["sentiment_trend"] = trend

    filter_df = filter_df[["date", "sentiment_cycle"]]
    sent = sent.drop(columns=["sentiment"])
    filter_df = filter_df.rename(columns={"sentiment_cycle": "sentiment"})
    sent = pd.merge(sent, filter_df, on="date", how="outer")
    sent = sent.groupby("date").mean().reset_index()

    sent = sent.rename(columns={"sentiment": url_map["document"][i]})
    sentiment = pd.merge(sentiment, sent, how="outer", left_on="date", right_on="date")

In [ ]:
fedfunds = pd.read_csv(
    "/Users/kylenabors/Documents/Database/Market Data/Fed Funds/Fed Funds.csv"
)
fedfunds["date"] = pd.to_datetime(fedfunds["date"])
sentiment = pd.merge(sentiment, fedfunds, how="left", left_on="date", right_on="date")

In [ ]:
stocks = sentiment.copy(deep=True)
stocks = stocks[["date", "sp500_return", "stoxx"]]
stocks["log_returns"] = np.log(1 + stocks["sp500_return"])
stocks["log_returns_eu"] = np.log(1 + stocks["stoxx"])
stocks = stocks.resample("Q", on="date").sum().reset_index()

for i in range(0, 6):
    stocks[f"log_returns_{i}"] = stocks["log_returns"].shift(i)
    stocks[f"log_returns_eu_{i}"] = stocks["log_returns_eu"].shift(i)

In [ ]:
# Resample the final 'sentiment' DataFrame to quarterly frequency
sentiment = sentiment.drop(columns=["sp500_return", "stoxx"])
sentiment = sentiment.resample("Q", on="date").mean().reset_index()

In [ ]:
# merging FSI data
sentiment["fsi"] = (
    sentiment["credit"]
    + sentiment["equity_valuation"]
    + sentiment["safe_assets"]
    + sentiment["funding"]
    + sentiment["volatility"]
)

In [ ]:
# Create lagged columns
for column in sentiment.columns:
    if column != "date":  # Skip the 'date' column
        sentiment[f"{column}_1"] = sentiment[column].shift(1)

for i in range(1, 5):
    sentiment[f"fed_minutes_forward_{i}"] = sentiment["fed_minutes"].shift(-i)
    sentiment[f"fed_minutes_lag_{i}"] = sentiment["fed_minutes"].shift(i)
    sentiment[f"press_conferences_forward_{i}"] = sentiment["press_conferences"].shift(
        -i
    )
    sentiment[f"credit_lag_{i}"] = sentiment["credit"].shift(i)
    sentiment[f"equity_valuation_lag_{i}"] = sentiment["equity_valuation"].shift(i)
    sentiment[f"safe_assets_lag_{i}"] = sentiment["safe_assets"].shift(i)
    sentiment[f"funding_lag_{i}"] = sentiment["funding"].shift(i)
    sentiment[f"volatility_lag_{i}"] = sentiment["volatility"].shift(i)
    sentiment[f"fsi_lag_{i}"] = sentiment["fsi"].shift(i)

In [ ]:
sentiment = pd.merge(sentiment, stocks, how="left", left_on="date", right_on="date")

In [ ]:
# sentiment = sentiment[
#     (sentiment["date"] >= "2000-01-01") & (sentiment["date"] <= "2023-01-30")
# ]

sentiment = sentiment[
    (sentiment["date"] >= "2000-06-01") & (sentiment["date"] <= "2022-06-30")
]

In [ ]:
# Export sentiment as csv
sentiment.to_csv(f"{database}/Sentiment.csv", index=False)

In [ ]:
print(sentiment.columns)

In [ ]:
import matplotlib.pyplot as plt

# List of columns containing minute sentiment for all banks
minute_sentiment_columns = [
    "fed_minutes",
    "ecb_minutes",
    "boe_minutes",
    "boj_minutes",
    "boa_minutes",
    "boc_minutes",
    "boswiss_minutes",
    "bosweaden",
]

# Plotting the minute sentiment for all banks
plt.figure(figsize=(14, 7))

for column in minute_sentiment_columns:
    if column in sentiment.columns:
        plt.plot(sentiment["date"], sentiment[column], label=column)

plt.xlabel("Date")
plt.ylabel("Minute Sentiment")
plt.title("Minute Sentiment for All Banks")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# List of columns containing speech sentiment for all banks
speech_sentiment_columns = [
    'fed_speeches', 
    'ecb_speeches', 
    'boe_speeches', 
    'boj_speeches', 
    'boa_speeches', 
    'boc_speeches', 
    'boswiss_speeches', 
    'bosweden_speeches',
]

# Define the layout of the subplots
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(20, 20))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot each sentiment column in its own subplot
for i, column in enumerate(speech_sentiment_columns):
    if column in sentiment.columns:
        axes[i].plot(sentiment['date'], sentiment[column], label=column)
        axes[i].set_xlabel('Date')
        axes[i].set_ylabel('Speech Sentiment')
        axes[i].set_title(column)
        axes[i].set_ylim(-0.25, 0.25)  # Set y-axis limits
        axes[i].legend()
        axes[i].grid(True)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Table 3 Pt. 1

x = [
    "press_conferences",
    "press_conferences_1",
    "fed_minutes_1",
]

yvar0 = "fed_minutes"
yvar1 = "fed_minutes_forward_1"
yvar2 = "fed_minutes_forward_2"
yvar3 = "fed_minutes_forward_3"
yvar4 = "fed_minutes_forward_4"


# Prepare the exogenous variables for each regression
exog0 = sm.add_constant(sentiment[x]).dropna()
exog1 = sm.add_constant(sentiment[x]).dropna()
exog2 = sm.add_constant(sentiment[x]).dropna()
exog3 = sm.add_constant(sentiment[x]).dropna()
exog4 = sm.add_constant(sentiment[x]).dropna()


# Fit the regression models
reg0 = sm.OLS(endog=sentiment[yvar0].loc[exog0.index], exog=exog0).fit(cov_type="HC0")
reg1 = sm.OLS(endog=sentiment[yvar1].loc[exog1.index], exog=exog1).fit(cov_type="HC0")
reg2 = sm.OLS(endog=sentiment[yvar2].loc[exog2.index], exog=exog2).fit(cov_type="HC0")
reg3 = sm.OLS(endog=sentiment[yvar3].loc[exog3.index], exog=exog3).fit(cov_type="HC0")
reg4 = sm.OLS(endog=sentiment[yvar4].loc[exog4.index], exog=exog4).fit(cov_type="HC0")


# Create the Stargazer object
stargazer = Stargazer([reg0, reg1, reg2, reg3, reg4])

# Customize the Stargazer output
stargazer.show_model_numbers(False)
stargazer.significant_digits(3)

stargazer.title("Fed Minute Sentiment Regressed on Financial Variables")
stargazer.custom_columns(
    [
        "Fed Sent. (t)",
        "Fed Sent. (t+1)",
        "Fed Sent. (t+2)",
        "Fed Sent. (t+3)",
        "Fed Sent. (t+4)",
    ]
)

stargazer.covariate_order(
    [
        "press_conferences",
        "press_conferences_1",
        "fed_minutes_1",
    ]
)

stargazer.rename_covariates(
    {
        "press_conferences": "ECB Sent (t)",
        "press_conferences_1": "ECB Sent (t-1)",
        "fed_minutes_1": "Fed Sent (t-1)",
    }
)
# Display the Stargazer output
display(HTML(stargazer.render_html()))

# Modify the LaTeX output to remove all \\[-1.8ex]
latex_output = stargazer.render_latex().replace("\\[-1.8ex]", "")
latex_output = latex_output.replace("!htbp", "H")
print(latex_output)

In [ ]:
# Table 3 Pt. 2

x = [
    "fed_minutes",
    "fed_minutes_1",
    "press_conferences_1",
]

yvar0 = "press_conferences"
yvar1 = "press_conferences_forward_1"
yvar2 = "press_conferences_forward_2"
yvar3 = "press_conferences_forward_3"
yvar4 = "press_conferences_forward_4"


# Prepare the exogenous variables for each regression
exog0 = sm.add_constant(sentiment[x]).dropna()
exog1 = sm.add_constant(sentiment[x]).dropna()
exog2 = sm.add_constant(sentiment[x]).dropna()
exog3 = sm.add_constant(sentiment[x]).dropna()
exog4 = sm.add_constant(sentiment[x]).dropna()


# Fit the regression models
reg0 = sm.OLS(endog=sentiment[yvar0].loc[exog0.index], exog=exog0).fit(cov_type="HC0")
reg1 = sm.OLS(endog=sentiment[yvar1].loc[exog1.index], exog=exog1).fit(cov_type="HC0")
reg2 = sm.OLS(endog=sentiment[yvar2].loc[exog2.index], exog=exog2).fit(cov_type="HC0")
reg3 = sm.OLS(endog=sentiment[yvar3].loc[exog3.index], exog=exog3).fit(cov_type="HC0")
reg4 = sm.OLS(endog=sentiment[yvar4].loc[exog4.index], exog=exog4).fit(cov_type="HC0")


# Create the Stargazer object
stargazer = Stargazer([reg0, reg1, reg2, reg3, reg4])

# Customize the Stargazer output
stargazer.show_model_numbers(False)
stargazer.significant_digits(3)

stargazer.title("Fed Minute Sentiment Regressed on Financial Variables")
stargazer.custom_columns(
    [
        "ECB Sent. (t)",
        "ECB Sent. (t+1)",
        "ECB Sent. (t+2)",
        "ECB Sent. (t+3)",
        "ECB Sent. (t+4)",
    ]
)

stargazer.covariate_order(
    [
        "fed_minutes",
        "fed_minutes_1",
        "press_conferences_1",
    ]
)

stargazer.rename_covariates(
    {
        "fed_minutes": "Fed Sent (t)",
        "fed_minutes_1": "Fed Sent (t-1)",
        "press_conferences_1": "ECB Sent (t-1)",
    }
)
# Display the Stargazer output
display(HTML(stargazer.render_html()))

# Modify the LaTeX output to remove all \\[-1.8ex]
latex_output = stargazer.render_latex().replace("\\[-1.8ex]", "")
latex_output = latex_output.replace("!htbp", "H")
print(latex_output)

In [ ]:
# # Define the independent variables for each regression
# x = ["fed_minutes", "fed_minutes_1", "fedfunds_1"]

# yvar0 = "fedfunds"
# yvar1 = "fedfunds_forward_1"
# yvar2 = "fedfunds_forward_2"
# yvar3 = "fedfunds_forward_3"
# yvar4 = "fedfunds_forward_4"


# # Prepare the exogenous variables for each regression
# exog0 = sm.add_constant(sentiment[x]).dropna()
# exog1 = sm.add_constant(sentiment[x]).dropna()
# exog2 = sm.add_constant(sentiment[x]).dropna()
# exog3 = sm.add_constant(sentiment[x]).dropna()
# exog4 = sm.add_constant(sentiment[x]).dropna()


# # Fit the regression models
# reg0 = sm.OLS(endog=sentiment[yvar0].loc[exog0.index], exog=exog0).fit(cov_type="HC0")
# reg1 = sm.OLS(endog=sentiment[yvar1].loc[exog1.index], exog=exog1).fit(cov_type="HC0")
# reg2 = sm.OLS(endog=sentiment[yvar2].loc[exog2.index], exog=exog2).fit(cov_type="HC0")
# reg3 = sm.OLS(endog=sentiment[yvar3].loc[exog3.index], exog=exog3).fit(cov_type="HC0")
# reg4 = sm.OLS(endog=sentiment[yvar4].loc[exog4.index], exog=exog4).fit(cov_type="HC0")


# # Create the Stargazer object
# stargazer = Stargazer([reg0, reg1, reg2, reg3, reg4])

# # Customize the Stargazer output
# stargazer.show_model_numbers(False)
# stargazer.significant_digits(3)

# # Display the Stargazer output
# display(HTML(stargazer.render_html()))
# print(stargazer.render_latex())

In [ ]:
# Fed SR LR ECB SR LR

x0 = [
    "higher4_5y5y_us",
    "higher4_5y5y_us_1",
    "lower0_5y5y_us",
    "lower0_5y5y_us_1",
    "fed_minutes_1",
]

x1 = [
    "zc_higher4_5y_us",
    "zc_higher4_5y_us_1",
    "zc_lower0_5y_us",
    "zc_lower0_5y_us_1",
    "fed_minutes_1",
]

x2 = [
    "higher4_5y5y_eu",
    "higher4_5y5y_eu_1",
    "lower0_5y5y_eu",
    "lower0_5y5y_eu_1",
    "press_conferences_1",
]

x3 = [
    "zc_higher4_5y_eu",
    "zc_higher4_5y_eu_1",
    "zc_lower0_5y_eu",
    "zc_lower0_5y_eu_1",
    "press_conferences_1",
]


yvar = "fed_minutes"
yvar1 = "fed_minutes"
yvar2 = "press_conferences"
yvar3 = "press_conferences"

exog0 = (sm.add_constant(sentiment[x0])).dropna()
exog1 = (sm.add_constant(sentiment[x1])).dropna()
exog2 = (sm.add_constant(sentiment[x2])).dropna()
exog3 = (sm.add_constant(sentiment[x3])).dropna()

reg0 = sm.OLS(endog=sentiment[yvar].loc[exog0.index], exog=exog0).fit(cov_type="HC0")
reg1 = sm.OLS(endog=sentiment[yvar1].loc[exog1.index], exog=exog1).fit(cov_type="HC0")
reg2 = sm.OLS(endog=sentiment[yvar2].loc[exog2.index], exog=exog2).fit(cov_type="HC0")
reg3 = sm.OLS(endog=sentiment[yvar3].loc[exog3.index], exog=exog3).fit(cov_type="HC0")

stargazer = Stargazer([reg0, reg1, reg2, reg3])
stargazer.title(
    "Regressing Fed Sentiment on SP500 Returns and Regressing ECB Sentiment on STOXX 600 Returns"
)
stargazer.custom_columns(["SR Fed(t)", "LR Fed(t)", "SR ECB(t)", "LR ECB(t)"])
stargazer.show_model_numbers(False)
stargazer.significant_digits(3)
stargazer.covariate_order(
    [
        "higher4_5y5y_us",
        "higher4_5y5y_us_1",
        "lower0_5y5y_us",
        "lower0_5y5y_us_1",
        "zc_higher4_5y_us",
        "zc_higher4_5y_us_1",
        "zc_lower0_5y_us",
        "zc_lower0_5y_us_1",
        "higher4_5y5y_eu",
        "higher4_5y5y_eu_1",
        "lower0_5y5y_eu",
        "lower0_5y5y_eu_1",
        "zc_higher4_5y_eu",
        "zc_higher4_5y_eu_1",
        "zc_lower0_5y_eu",
        "zc_lower0_5y_eu_1",
        "fed_minutes_1",
        "press_conferences_1",
    ]
)
stargazer.rename_covariates(
    {
        "higher4_5y5y_us": "Inflation(t)",
        "higher4_5y5y_us_1": "Inflation(t-1)",
        "lower0_5y5y_us": "Deflation(t)",
        "lower0_5y5y_us_1": "Deflation(t-1)",
        "zc_higher4_5y_us": "Inflation(t)",
        "zc_higher4_5y_us_1": "Inflation(t-1)",
        "zc_lower0_5y_us": "Deflation(t)",
        "zc_lower0_5y_us_1": "Deflation(t-1)",
        "higher4_5y5y_eu": "Inflation(t)",
        "higher4_5y5y_eu_1": "Inflation(t-1)",
        "lower0_5y5y_eu": "Deflation(t)",
        "lower0_5y5y_eu_1": "Deflation(t-1)",
        "zc_higher4_5y_eu": "Inflation(t)",
        "zc_higher4_5y_eu_1": "Inflation(t-1)",
        "zc_lower0_5y_eu": "Deflation(t)",
        "zc_lower0_5y_eu_1": "Deflation(t)",
        "fed_minutes_1": "Fed(t-1)",
        "press_conferences_1": "ECB(t-1)",
    },
)
stargazer.add_custom_notes(
    [
        "The unit for t is the time between a given meeting and the previous meeting for the given Central Bank."
    ]
)
# Display the Stargazer output
display(HTML(stargazer.render_html()))

# Modify the LaTeX output to remove all \\[-1.8ex]
latex_output = stargazer.render_latex().replace("\\[-1.8ex]", "")
latex_output = latex_output.replace("!htbp", "H")
print(latex_output)

In [ ]:
# Fed SR LR ECB SR LR

x0 = [
    "higher4_5y5y_us",
    "higher4_5y5y_us_1",
    "lower0_5y5y_us",
    "lower0_5y5y_us_1",
    "fed_minutes_1",
    "log_returns_0",
    "log_returns_1",
    "log_returns_2",
    "log_returns_3",
    "log_returns_4",
    "log_returns_5",
    "12m_inf",
    "12m_inf_1",
]

x1 = [
    "zc_higher4_5y_us",
    "zc_higher4_5y_us_1",
    "zc_lower0_5y_us",
    "zc_lower0_5y_us_1",
    "fed_minutes_1",
    "log_returns_0",
    "log_returns_1",
    "log_returns_2",
    "log_returns_3",
    "log_returns_4",
    "log_returns_5",
    "12m_inf",
    "12m_inf_1",
]

x2 = [
    "higher4_5y5y_eu",
    "higher4_5y5y_eu_1",
    "lower0_5y5y_eu",
    "lower0_5y5y_eu_1",
    "press_conferences_1",
    "log_returns_eu_0",
    "log_returns_eu_1",
    "log_returns_eu_2",
    "log_returns_eu_3",
    "log_returns_eu_4",
    "log_returns_eu_5",
]

x3 = [
    "zc_higher4_5y_eu",
    "zc_higher4_5y_eu_1",
    "zc_lower0_5y_eu",
    "zc_lower0_5y_eu_1",
    "press_conferences_1",
    "log_returns_eu_0",
    "log_returns_eu_1",
    "log_returns_eu_2",
    "log_returns_eu_3",
    "log_returns_eu_4",
    "log_returns_eu_5",
]


yvar = "fed_minutes"
yvar1 = "fed_minutes"
yvar2 = "press_conferences"
yvar3 = "press_conferences"

exog0 = (sm.add_constant(sentiment[x0])).dropna()
exog1 = (sm.add_constant(sentiment[x1])).dropna()
exog2 = (sm.add_constant(sentiment[x2])).dropna()
exog3 = (sm.add_constant(sentiment[x3])).dropna()

reg0 = sm.OLS(endog=sentiment[yvar].loc[exog0.index], exog=exog0).fit(cov_type="HC0")
reg1 = sm.OLS(endog=sentiment[yvar1].loc[exog1.index], exog=exog1).fit(cov_type="HC0")
reg2 = sm.OLS(endog=sentiment[yvar2].loc[exog2.index], exog=exog2).fit(cov_type="HC0")
reg3 = sm.OLS(endog=sentiment[yvar3].loc[exog3.index], exog=exog3).fit(cov_type="HC0")

stargazer = Stargazer([reg0, reg1, reg2, reg3])
stargazer.title(
    "Regressing Fed Sentiment on SP500 Returns and Regressing ECB Sentiment on STOXX 600 Returns"
)
stargazer.custom_columns(["Fed(t)", "Fed(t)", "ECB(t)", "ECB(t)"])
stargazer.show_model_numbers(False)
stargazer.significant_digits(3)
stargazer.covariate_order(
    [
        "12m_inf",
        "12m_inf_1",
        "higher4_5y5y_us",
        "higher4_5y5y_us_1",
        "lower0_5y5y_us",
        "lower0_5y5y_us_1",
        "zc_higher4_5y_us",
        "zc_higher4_5y_us_1",
        "zc_lower0_5y_us",
        "zc_lower0_5y_us_1",
        "higher4_5y5y_eu",
        "higher4_5y5y_eu_1",
        "lower0_5y5y_eu",
        "lower0_5y5y_eu_1",
        "zc_higher4_5y_eu",
        "zc_higher4_5y_eu_1",
        "zc_lower0_5y_eu",
        "zc_lower0_5y_eu_1",
        "log_returns_0",
        "log_returns_1",
        "log_returns_2",
        "log_returns_3",
        "log_returns_4",
        "log_returns_5",
        "log_returns_eu_0",
        "log_returns_eu_1",
        "log_returns_eu_2",
        "log_returns_eu_3",
        "log_returns_eu_4",
        "log_returns_eu_5",
        "fed_minutes_1",
        "press_conferences_1",
    ]
)
stargazer.rename_covariates(
    {
        "12m_inf": "Inflation(t)",
        "12m_inf_1": "Inflation(t-1)",
        "higher4_5y5y_us": "Inflation Proj. SR(t)",
        "higher4_5y5y_us_1": "Inflation Proj. SR(t-1)",
        "lower0_5y5y_us": "Deflation Proj. SR(t)",
        "lower0_5y5y_us_1": "Deflation Proj. SR(t-1)",
        "zc_higher4_5y_us": "Inflation Proj. LR(t)",
        "zc_higher4_5y_us_1": "Inflation Proj. LR(t-1)",
        "zc_lower0_5y_us": "Deflation Proj. LR(t)",
        "zc_lower0_5y_us_1": "Deflation Proj. LR(t-1)",
        "higher4_5y5y_eu": "Inflation Proj. SR(t)",
        "higher4_5y5y_eu_1": "Inflation Proj. SR(t-1)",
        "lower0_5y5y_eu": "Deflation Proj. SR(t)",
        "lower0_5y5y_eu_1": "Deflation Proj. SR(t-1)",
        "zc_higher4_5y_eu": "Inflation Proj. LR(t)",
        "zc_higher4_5y_eu_1": "Inflation Proj. LR(t-1)",
        "zc_lower0_5y_eu": "Deflation Proj. LR(t)",
        "zc_lower0_5y_eu_1": "Deflation Proj. LR(t)",
        "log_returns_0": "SP00 Returns(t)",
        "log_returns_1": "SP00 Returns(t-1)",
        "log_returns_2": "SP00 Returns(t-2)",
        "log_returns_3": "SP00 Returns(t-3)",
        "log_returns_4": "SP00 Returns(t-4)",
        "log_returns_5": "SP00 Returns(t-5)",
        "fed_minutes_1": "Fed(t-1)",
        "log_returns_eu_0": "STOXX600 Returns(t)",
        "log_returns_eu_1": "STOXX600 Returns(t-1)",
        "log_returns_eu_2": "STOXX600 Returns(t-2)",
        "log_returns_eu_3": "STOXX600 Returns(t-3)",
        "log_returns_eu_4": "STOXX600 Returns(t-4)",
        "log_returns_eu_5": "STOXX600 Returns(t-5)",
        "press_conferences_1": "ECB(t-1)",
    },
)
stargazer.add_custom_notes(
    [
        "The unit for t is the time between a given meeting and the previous meeting for the given Central Bank."
    ]
)
# Display the Stargazer output
display(HTML(stargazer.render_html()))

# Modify the LaTeX output to remove all \\[-1.8ex]
latex_output = stargazer.render_latex().replace("\\[-1.8ex]", "")
latex_output = latex_output.replace("!htbp", "H")
print(latex_output)

In [ ]:
# Define the independent variables for each regression
x = [
    "fsi",
    "fsi_1",
    "fed_minutes_1",
]

yvar0 = "fed_minutes"
yvar1 = "fed_minutes_forward_1"
yvar2 = "fed_minutes_forward_2"
yvar3 = "fed_minutes_forward_3"
yvar4 = "fed_minutes_forward_4"


# Prepare the exogenous variables for each regression
exog0 = sm.add_constant(sentiment[x]).dropna()
exog1 = sm.add_constant(sentiment[x]).dropna()
exog2 = sm.add_constant(sentiment[x]).dropna()
exog3 = sm.add_constant(sentiment[x]).dropna()
exog4 = sm.add_constant(sentiment[x]).dropna()


# Fit the regression models
reg0 = sm.OLS(endog=sentiment[yvar0].loc[exog0.index], exog=exog0).fit(cov_type="HC0")
reg1 = sm.OLS(endog=sentiment[yvar1].loc[exog1.index], exog=exog1).fit(cov_type="HC0")
reg2 = sm.OLS(endog=sentiment[yvar2].loc[exog2.index], exog=exog2).fit(cov_type="HC0")
reg3 = sm.OLS(endog=sentiment[yvar3].loc[exog3.index], exog=exog3).fit(cov_type="HC0")
reg4 = sm.OLS(endog=sentiment[yvar4].loc[exog4.index], exog=exog4).fit(cov_type="HC0")


# Create the Stargazer object
stargazer = Stargazer([reg0, reg1, reg2, reg3, reg4])

# Customize the Stargazer output
stargazer.show_model_numbers(False)
stargazer.significant_digits(3)

stargazer.title("Fed Minute Sentiment Regressed on Financial Sentiment Index")
stargazer.custom_columns(
    [
        "Fed Sent. (t)",
        "Fed Sent. (t+1)",
        "Fed Sent. (t+2)",
        "Fed Sent. (t+3)",
        "Fed Sent. (t+4)",
    ]
)

stargazer.covariate_order(
    [
        "fsi",
        "fsi_1",
        "fed_minutes_1",
    ]
)

stargazer.rename_covariates(
    {
        "fsi": "Financial Sentiment Index",
        "fsi_1": "Financial Sentiment Index (t-1)",
        "fed_minutes_1": "Fed Sent. (t-1)",
    }
)
# Display the Stargazer output
display(HTML(stargazer.render_html()))

# Modify the LaTeX output to remove all \\[-1.8ex]
latex_output = stargazer.render_latex().replace("\\[-1.8ex]", "")
latex_output = latex_output.replace("!htbp", "H")
print(latex_output)

In [ ]:
# Define the independent variables for each regression
x = [
    "credit",
    "credit_1",
    "fed_minutes_1",
]

yvar0 = "fed_minutes"
yvar1 = "fed_minutes_forward_1"
yvar2 = "fed_minutes_forward_2"
yvar3 = "fed_minutes_forward_3"
yvar4 = "fed_minutes_forward_4"


# Prepare the exogenous variables for each regression
exog0 = sm.add_constant(sentiment[x]).dropna()
exog1 = sm.add_constant(sentiment[x]).dropna()
exog2 = sm.add_constant(sentiment[x]).dropna()
exog3 = sm.add_constant(sentiment[x]).dropna()
exog4 = sm.add_constant(sentiment[x]).dropna()


# Fit the regression models
reg0 = sm.OLS(endog=sentiment[yvar0].loc[exog0.index], exog=exog0).fit(cov_type="HC0")
reg1 = sm.OLS(endog=sentiment[yvar1].loc[exog1.index], exog=exog1).fit(cov_type="HC0")
reg2 = sm.OLS(endog=sentiment[yvar2].loc[exog2.index], exog=exog2).fit(cov_type="HC0")
reg3 = sm.OLS(endog=sentiment[yvar3].loc[exog3.index], exog=exog3).fit(cov_type="HC0")
reg4 = sm.OLS(endog=sentiment[yvar4].loc[exog4.index], exog=exog4).fit(cov_type="HC0")


# Create the Stargazer object
stargazer = Stargazer([reg0, reg1, reg2, reg3, reg4])

# Customize the Stargazer output
stargazer.show_model_numbers(False)
stargazer.significant_digits(3)

stargazer.title("Fed Minute Sentiment Regressed on Financial Sentiment Index Credit")
stargazer.custom_columns(
    [
        "Fed Sent. (t)",
        "Fed Sent. (t+1)",
        "Fed Sent. (t+2)",
        "Fed Sent. (t+3)",
        "Fed Sent. (t+4)",
    ]
)

stargazer.covariate_order(
    [
        "credit",
        "credit_1",
        "fed_minutes_1",
    ]
)

stargazer.rename_covariates(
    {
        "credit": "Credit",
        "credit_1": "Credit (t-1)",
        "fed_minutes_1": "Fed Sent. (t-1)",
    }
)
# Display the Stargazer output
display(HTML(stargazer.render_html()))

# Modify the LaTeX output to remove all \\[-1.8ex]
latex_output = stargazer.render_latex().replace("\\[-1.8ex]", "")
latex_output = latex_output.replace("!htbp", "H")
print(latex_output)

In [ ]:
# Define the independent variables for each regression
x = [
    "equity_valuation",
    "equity_valuation_1",
    "fed_minutes_1",
]

yvar0 = "fed_minutes"
yvar1 = "fed_minutes_forward_1"
yvar2 = "fed_minutes_forward_2"
yvar3 = "fed_minutes_forward_3"
yvar4 = "fed_minutes_forward_4"


# Prepare the exogenous variables for each regression
exog0 = sm.add_constant(sentiment[x]).dropna()
exog1 = sm.add_constant(sentiment[x]).dropna()
exog2 = sm.add_constant(sentiment[x]).dropna()
exog3 = sm.add_constant(sentiment[x]).dropna()
exog4 = sm.add_constant(sentiment[x]).dropna()


# Fit the regression models
reg0 = sm.OLS(endog=sentiment[yvar0].loc[exog0.index], exog=exog0).fit(cov_type="HC0")
reg1 = sm.OLS(endog=sentiment[yvar1].loc[exog1.index], exog=exog1).fit(cov_type="HC0")
reg2 = sm.OLS(endog=sentiment[yvar2].loc[exog2.index], exog=exog2).fit(cov_type="HC0")
reg3 = sm.OLS(endog=sentiment[yvar3].loc[exog3.index], exog=exog3).fit(cov_type="HC0")
reg4 = sm.OLS(endog=sentiment[yvar4].loc[exog4.index], exog=exog4).fit(cov_type="HC0")


# Create the Stargazer object
stargazer = Stargazer([reg0, reg1, reg2, reg3, reg4])

# Customize the Stargazer output
stargazer.show_model_numbers(False)
stargazer.significant_digits(3)

stargazer.title(
    "Fed Minute Sentiment Regressed on Financial Sentiment Index Equity Valuation"
)
stargazer.custom_columns(
    [
        "Fed Sent. (t)",
        "Fed Sent. (t+1)",
        "Fed Sent. (t+2)",
        "Fed Sent. (t+3)",
        "Fed Sent. (t+4)",
    ]
)

stargazer.covariate_order(
    [
        "equity_valuation",
        "equity_valuation_1",
        "fed_minutes_1",
    ]
)

stargazer.rename_covariates(
    {
        "equity_valuation": "Equity Valuation",
        "equity_valuation_1": "Equity Valuation (t-1)",
        "fed_minutes_1": "Fed Sent. (t-1)",
    }
)
# Display the Stargazer output
display(HTML(stargazer.render_html()))

# Modify the LaTeX output to remove all \\[-1.8ex]
latex_output = stargazer.render_latex().replace("\\[-1.8ex]", "")
latex_output = latex_output.replace("!htbp", "H")
print(latex_output)

In [ ]:
# Define the independent variables for each regression
x = [
    "safe_assets",
    "safe_assets_1",
    "fed_minutes_1",
]

yvar0 = "fed_minutes"
yvar1 = "fed_minutes_forward_1"
yvar2 = "fed_minutes_forward_2"
yvar3 = "fed_minutes_forward_3"
yvar4 = "fed_minutes_forward_4"


# Prepare the exogenous variables for each regression
exog0 = sm.add_constant(sentiment[x]).dropna()
exog1 = sm.add_constant(sentiment[x]).dropna()
exog2 = sm.add_constant(sentiment[x]).dropna()
exog3 = sm.add_constant(sentiment[x]).dropna()
exog4 = sm.add_constant(sentiment[x]).dropna()


# Fit the regression models
reg0 = sm.OLS(endog=sentiment[yvar0].loc[exog0.index], exog=exog0).fit(cov_type="HC0")
reg1 = sm.OLS(endog=sentiment[yvar1].loc[exog1.index], exog=exog1).fit(cov_type="HC0")
reg2 = sm.OLS(endog=sentiment[yvar2].loc[exog2.index], exog=exog2).fit(cov_type="HC0")
reg3 = sm.OLS(endog=sentiment[yvar3].loc[exog3.index], exog=exog3).fit(cov_type="HC0")
reg4 = sm.OLS(endog=sentiment[yvar4].loc[exog4.index], exog=exog4).fit(cov_type="HC0")


# Create the Stargazer object
stargazer = Stargazer([reg0, reg1, reg2, reg3, reg4])

# Customize the Stargazer output
stargazer.show_model_numbers(False)
stargazer.significant_digits(3)

stargazer.title(
    "Fed Minute Sentiment Regressed on Financial Sentiment Index Safe Assets"
)
stargazer.custom_columns(
    [
        "Fed Sent. (t)",
        "Fed Sent. (t+1)",
        "Fed Sent. (t+2)",
        "Fed Sent. (t+3)",
        "Fed Sent. (t+4)",
    ]
)

stargazer.covariate_order(
    [
        "safe_assets",
        "safe_assets_1",
        "fed_minutes_1",
    ]
)

stargazer.rename_covariates(
    {
        "safe_assets": "Safe Assets",
        "safe_assets_1": "Safe Assets (t-1)",
        "fed_minutes_1": "Fed Sent. (t-1)",
    }
)
# Display the Stargazer output
display(HTML(stargazer.render_html()))

# Modify the LaTeX output to remove all \\[-1.8ex]
latex_output = stargazer.render_latex().replace("\\[-1.8ex]", "")
latex_output = latex_output.replace("!htbp", "H")
print(latex_output)

In [ ]:
# Define the independent variables for each regression
x = [
    "funding",
    "funding_1",
    "fed_minutes_1",
]

yvar0 = "fed_minutes"
yvar1 = "fed_minutes_forward_1"
yvar2 = "fed_minutes_forward_2"
yvar3 = "fed_minutes_forward_3"
yvar4 = "fed_minutes_forward_4"


# Prepare the exogenous variables for each regression
exog0 = sm.add_constant(sentiment[x]).dropna()
exog1 = sm.add_constant(sentiment[x]).dropna()
exog2 = sm.add_constant(sentiment[x]).dropna()
exog3 = sm.add_constant(sentiment[x]).dropna()
exog4 = sm.add_constant(sentiment[x]).dropna()


# Fit the regression models
reg0 = sm.OLS(endog=sentiment[yvar0].loc[exog0.index], exog=exog0).fit(cov_type="HC0")
reg1 = sm.OLS(endog=sentiment[yvar1].loc[exog1.index], exog=exog1).fit(cov_type="HC0")
reg2 = sm.OLS(endog=sentiment[yvar2].loc[exog2.index], exog=exog2).fit(cov_type="HC0")
reg3 = sm.OLS(endog=sentiment[yvar3].loc[exog3.index], exog=exog3).fit(cov_type="HC0")
reg4 = sm.OLS(endog=sentiment[yvar4].loc[exog4.index], exog=exog4).fit(cov_type="HC0")


# Create the Stargazer object
stargazer = Stargazer([reg0, reg1, reg2, reg3, reg4])

# Customize the Stargazer output
stargazer.show_model_numbers(False)
stargazer.significant_digits(3)

stargazer.title("Fed Minute Sentiment Regressed on Financial Sentiment Index Funding")
stargazer.custom_columns(
    [
        "Fed Sent. (t)",
        "Fed Sent. (t+1)",
        "Fed Sent. (t+2)",
        "Fed Sent. (t+3)",
        "Fed Sent. (t+4)",
    ]
)

stargazer.covariate_order(
    [
        "funding",
        "funding_1",
        "fed_minutes_1",
    ]
)

stargazer.rename_covariates(
    {
        "funding": "Funding",
        "funding_1": "Funding (t-1)",
        "fed_minutes_1": "Fed Sent. (t-1)",
    }
)
# Display the Stargazer output
display(HTML(stargazer.render_html()))

# Modify the LaTeX output to remove all \\[-1.8ex]
latex_output = stargazer.render_latex().replace("\\[-1.8ex]", "")
latex_output = latex_output.replace("!htbp", "H")
print(latex_output)

In [ ]:
# Define the independent variables for each regression
x = [
    "volatility",
    "volatility_1",
    "fed_minutes_1",
]

yvar0 = "fed_minutes"
yvar1 = "fed_minutes_forward_1"
yvar2 = "fed_minutes_forward_2"
yvar3 = "fed_minutes_forward_3"
yvar4 = "fed_minutes_forward_4"


# Prepare the exogenous variables for each regression
exog0 = sm.add_constant(sentiment[x]).dropna()
exog1 = sm.add_constant(sentiment[x]).dropna()
exog2 = sm.add_constant(sentiment[x]).dropna()
exog3 = sm.add_constant(sentiment[x]).dropna()
exog4 = sm.add_constant(sentiment[x]).dropna()


# Fit the regression models
reg0 = sm.OLS(endog=sentiment[yvar0].loc[exog0.index], exog=exog0).fit(cov_type="HC0")
reg1 = sm.OLS(endog=sentiment[yvar1].loc[exog1.index], exog=exog1).fit(cov_type="HC0")
reg2 = sm.OLS(endog=sentiment[yvar2].loc[exog2.index], exog=exog2).fit(cov_type="HC0")
reg3 = sm.OLS(endog=sentiment[yvar3].loc[exog3.index], exog=exog3).fit(cov_type="HC0")
reg4 = sm.OLS(endog=sentiment[yvar4].loc[exog4.index], exog=exog4).fit(cov_type="HC0")


# Create the Stargazer object
stargazer = Stargazer([reg0, reg1, reg2, reg3, reg4])

# Customize the Stargazer output
stargazer.show_model_numbers(False)
stargazer.significant_digits(3)

stargazer.title(
    "Fed Minute Sentiment Regressed on Financial Sentiment Index Volatility"
)
stargazer.custom_columns(
    [
        "Fed Sent. (t)",
        "Fed Sent. (t+1)",
        "Fed Sent. (t+2)",
        "Fed Sent. (t+3)",
        "Fed Sent. (t+4)",
    ]
)

stargazer.covariate_order(
    [
        "volatility",
        "volatility_1",
        "fed_minutes_1",
    ]
)

stargazer.rename_covariates(
    {
        "volatility": "Volatility",
        "volatility_1": "Volatility (t-1)",
        "fed_minutes_1": "Fed Sent. (t-1)",
    }
)
# Display the Stargazer output
display(HTML(stargazer.render_html()))

# Modify the LaTeX output to remove all \\[-1.8ex]
latex_output = stargazer.render_latex().replace("\\[-1.8ex]", "")
latex_output = latex_output.replace("!htbp", "H")
print(latex_output)

In [ ]:
# Define the independent variables for each regression
x = [
    "credit",
    "credit_1",
    "equity_valuation",
    "equity_valuation_1",
    "safe_assets",
    "safe_assets_1",
    "funding",
    "funding_1",
    "volatility",
    "volatility_1",
    "fed_minutes_1",
]

yvar0 = "fed_minutes"
yvar1 = "fed_minutes_forward_1"
yvar2 = "fed_minutes_forward_2"
yvar3 = "fed_minutes_forward_3"
yvar4 = "fed_minutes_forward_4"


# Prepare the exogenous variables for each regression
exog0 = sm.add_constant(sentiment[x]).dropna()
exog1 = sm.add_constant(sentiment[x]).dropna()
exog2 = sm.add_constant(sentiment[x]).dropna()
exog3 = sm.add_constant(sentiment[x]).dropna()
exog4 = sm.add_constant(sentiment[x]).dropna()


# Fit the regression models
reg0 = sm.OLS(endog=sentiment[yvar0].loc[exog0.index], exog=exog0).fit(cov_type="HC0")
reg1 = sm.OLS(endog=sentiment[yvar1].loc[exog1.index], exog=exog1).fit(cov_type="HC0")
reg2 = sm.OLS(endog=sentiment[yvar2].loc[exog2.index], exog=exog2).fit(cov_type="HC0")
reg3 = sm.OLS(endog=sentiment[yvar3].loc[exog3.index], exog=exog3).fit(cov_type="HC0")
reg4 = sm.OLS(endog=sentiment[yvar4].loc[exog4.index], exog=exog4).fit(cov_type="HC0")


# Create the Stargazer object
stargazer = Stargazer([reg0, reg1, reg2, reg3, reg4])

# Customize the Stargazer output
stargazer.show_model_numbers(False)
stargazer.significant_digits(3)

stargazer.title("Fed Minute Sentiment Regressed on Financial Sentiment Index Variables")
stargazer.custom_columns(
    [
        "Fed Sent. (t)",
        "Fed Sent. (t+1)",
        "Fed Sent. (t+2)",
        "Fed Sent. (t+3)",
        "Fed Sent. (t+4)",
    ]
)

stargazer.covariate_order(
    [
        "credit",
        "credit_1",
        "equity_valuation",
        "equity_valuation_1",
        "safe_assets",
        "safe_assets_1",
        "funding",
        "funding_1",
        "volatility",
        "volatility_1",
        "fed_minutes_1",
    ]
)

stargazer.rename_covariates(
    {
        "credit": "Credit",
        "credit_1": "Credit (t-1)",
        "equity_valuation": "Equity Valuation",
        "equity_valuation_1": "Equity Valuation (t-1)",
        "safe_assets": "Safe Assets",
        "safe_assets_1": "Safe Assets (t-1)",
        "funding": "Funding",
        "funding_1": "Funding (t-1)",
        "volatility": "Volatility",
        "volatility_1": "Volatility (t-1)",
        "fed_minutes_1": "Fed Sent. (t-1)",
    }
)
# Display the Stargazer output
display(HTML(stargazer.render_html()))

# Modify the LaTeX output to remove all \\[-1.8ex]
latex_output = stargazer.render_latex().replace("\\[-1.8ex]", "")
latex_output = latex_output.replace("!htbp", "H")
print(latex_output)

In [ ]:
# Define the independent variables for each regression
x0 = [
    "fed_minutes",
    "fed_minutes_lag_1",
    "fed_minutes_lag_2",
    "fed_minutes_lag_3",
    "fed_minutes_lag_4",
    "credit_lag_1",
]

x1 = [
    "fed_minutes",
    "fed_minutes_lag_1",
    "fed_minutes_lag_2",
    "fed_minutes_lag_3",
    "fed_minutes_lag_4",
    "equity_valuation_lag_1",
]

x2 = [
    "fed_minutes",
    "fed_minutes_lag_1",
    "fed_minutes_lag_2",
    "fed_minutes_lag_3",
    "fed_minutes_lag_4",
    "safe_assets_lag_1",
]

x3 = [
    "fed_minutes",
    "fed_minutes_lag_1",
    "fed_minutes_lag_2",
    "fed_minutes_lag_3",
    "fed_minutes_lag_4",
    "funding_lag_1",
]

x4 = [
    "fed_minutes",
    "fed_minutes_lag_1",
    "fed_minutes_lag_2",
    "fed_minutes_lag_3",
    "fed_minutes_lag_4",
    "volatility_lag_1",
]


yvar0 = "credit"
yvar1 = "equity_valuation"
yvar2 = "safe_assets"
yvar3 = "funding"
yvar4 = "volatility"


# Prepare the exogenous variables for each regression
exog0 = sm.add_constant(sentiment[x0]).dropna()
exog1 = sm.add_constant(sentiment[x1]).dropna()
exog2 = sm.add_constant(sentiment[x2]).dropna()
exog3 = sm.add_constant(sentiment[x3]).dropna()
exog4 = sm.add_constant(sentiment[x4]).dropna()


# Fit the regression models
reg0 = sm.OLS(endog=sentiment[yvar0].loc[exog0.index], exog=exog0).fit(cov_type="HC0")
reg1 = sm.OLS(endog=sentiment[yvar1].loc[exog1.index], exog=exog1).fit(cov_type="HC0")
reg2 = sm.OLS(endog=sentiment[yvar2].loc[exog2.index], exog=exog2).fit(cov_type="HC0")
reg3 = sm.OLS(endog=sentiment[yvar3].loc[exog3.index], exog=exog3).fit(cov_type="HC0")
reg4 = sm.OLS(endog=sentiment[yvar4].loc[exog4.index], exog=exog4).fit(cov_type="HC0")


# Create the Stargazer object
stargazer = Stargazer([reg0, reg1, reg2, reg3, reg4])

# Customize the Stargazer output
stargazer.show_model_numbers(False)
stargazer.significant_digits(3)

stargazer.title("Financial Stress Index Regressed on Fed Minutes Sentiment")
stargazer.custom_columns(
    [
        "Credit (t)",
        "Equity Valuation (t)",
        "Safe Assets (t)",
        "Funding (t)",
        "Volatility (t)",
    ]
)

stargazer.covariate_order(
    [
        "fed_minutes",
        "fed_minutes_lag_1",
        "fed_minutes_lag_2",
        "fed_minutes_lag_3",
        "fed_minutes_lag_4",
        "credit_lag_1",
        "equity_valuation_lag_1",
        "safe_assets_lag_1",
        "funding_lag_1",
        "volatility_lag_1",
    ]
)

stargazer.rename_covariates(
    {
        "fed_minutes": "Fed Sentiment (t)",
        "fed_minutes_lag_1": "Fed Sentiment (t-1)",
        "fed_minutes_lag_2": "Fed Sentiment (t-2)",
        "fed_minutes_lag_3": "Fed Sentiment (t-3)",
        "fed_minutes_lag_4": "Fed Sentiment (t-4)",
        "credit_lag_1": "Credit (t-1)",
        "equity_valuation_lag_1": "Equity Valuation (t-1)",
        "safe_assets_lag_1": "Safe Assets (t-1)",
        "funding_lag_1": "Funding (t-1)",
        "volatility_lag_1": "Volatility (t-1)",
    }
)
# Display the Stargazer output
display(HTML(stargazer.render_html()))

# Modify the LaTeX output to remove all \\[-1.8ex]
latex_output = stargazer.render_latex().replace("\\[-1.8ex]", "")
latex_output = latex_output.replace("!htbp", "H")
print(latex_output)